In [9]:
import networkx as nx
import numpy as np
import pandas as pd 
import operator
import matplotlib.pyplot as plt
import random

In [2]:
nx_graphs = []
agg_graph = nx.MultiGraph()
venue_df_list = []
for i in range(31):
    # with open(f'..\ATX_Networks\sim_day_{i+1}.gexf') as graph_file:
    nx_graphs.append(nx.read_gexf(f'..\ATX_Networks\sim_day_{i+1}.gexf'))
    venue_df_list.append(nx.to_pandas_edgelist(nx_graphs[i],source="Source",target="Target"))

concat_venue_df = pd.concat(venue_df_list)
# concat_venue_df.columns = ['source','target','venueid']
concat_venue_df.head()


,Source,Target,time_spent,id,hour,venueid,visits
0,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,976,0,2,440da2cbf964a52091301fe3,18
1,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,cf4e564f4c8e3aecc9dacafce90cada3eabaff58464e74...,100,1,2,440da2cbf964a52091301fe3,2
2,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,594a39e074fda9219102eff386f3da1944494aaa18c036...,14,2,4,440da2cbf964a52091301fe3,2
3,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,a1470e73f1bc4f7c757c49138afd84bd1ad09e9277526a...,148,3,5,440da2cbf964a52091301fe3,3
4,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,6f8848838f9afe5a7b38c23ad5018179f69e518ee721ba...,526,4,5,440da2cbf964a52091301fe3,9


In [3]:
places_df = pd.read_csv("../RVF_ATX_PID_HZ_Places_Lookup.tsv",delimiter='\t')
merged_df = concat_venue_df.merge(places_df, on='venueid')
merged_df.head()

,Source,Target,time_spent,id,hour,venueid,visits,venuename,address,city,...,level1cat,level2catId,level2cat,level3catId,level3cat,chainid,chainname,parentvenueid,zip,sample
0,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,976,0,2,440da2cbf964a52091301fe3,18,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Travel & Transport,4bf58dd8d48988d1ed931735,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all
1,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,cf4e564f4c8e3aecc9dacafce90cada3eabaff58464e74...,100,1,2,440da2cbf964a52091301fe3,2,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Travel & Transport,4bf58dd8d48988d1ed931735,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all
2,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,594a39e074fda9219102eff386f3da1944494aaa18c036...,14,2,4,440da2cbf964a52091301fe3,2,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Travel & Transport,4bf58dd8d48988d1ed931735,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all
3,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,a1470e73f1bc4f7c757c49138afd84bd1ad09e9277526a...,148,3,5,440da2cbf964a52091301fe3,3,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Travel & Transport,4bf58dd8d48988d1ed931735,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all
4,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,6f8848838f9afe5a7b38c23ad5018179f69e518ee721ba...,526,4,5,440da2cbf964a52091301fe3,9,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Travel & Transport,4bf58dd8d48988d1ed931735,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all


In [11]:
#time_spent, hour, zip, visits, venueid
unique_venues = set(merged_df["venueid"])
unique_hours = np.arange(25)
max_time_spent = merged_df['time_spent'].max()
min_time_spent = merged_df['time_spent'].min()
max_visits = merged_df['visits'].max()
min_visits = merged_df['visits'].min()
time_spent_range = np.arange(min_time_spent, max_time_spent+1)
visit_range = np.arange(min_visits,max_visits+1)
print(f"The range of time spent is from {min_time_spent} to {max_time_spent}")


The range of time spent is from 0 to 3420


In [18]:
#Generate negative samples
source_set = set(merged_df['Source'].unique())
target_set = set(merged_df['Target'].unique())
node_set = source_set.union(target_set)
neg_df = pd.DataFrame(columns=["Source, Target, venueid, hour, time_spent, visits"])
for i in range(len(merged_df['Source'])):
    neg_edge = random.sample(node_set,k=2)
    if merged_df.loc[((merged_df['Source']!=neg_edge[0]) &
    (merged_df.loc(merged_df['Target'])!=neg_edge[1])) | 
    ((merged_df['Target']!=neg_edge[0]) &
    (merged_df.loc(merged_df['Source'])!=neg_edge[1]))]:
        sample_hour = random.sample(unique_hours,k=1)
        sample_time_spent = random.sample(time_spent_range,k=1)
        sample_venueid = random.sample(unique_venues, k=1)
        sample_visits = random.sample(visit_range,k=1)
        neg_df.append({
            "Source":neg_edge[0],
            "Target":neg_edge[1],
            "venueid":sample_venueid,
            "hour":sample_hour,
            "time_spent":sample_time_spent,
            "visits":sample_visits
        }, ignore_index=True)


TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [4]:

size = len(node_set)
node_encodings = np.arange(size)
node_list = list(node_set)



In [5]:
node_encoding_df = pd.DataFrame({'Node':node_list,'Source_Encoding':node_encodings})
node_encoding_df.shape
encoded_df = merged_df.merge(node_encoding_df, left_on='Source',right_on='Node')
node_encoding_df = pd.DataFrame({'Node':node_list,'Target_Encoding':node_encodings})
encoded_df = encoded_df.merge(node_encoding_df, left_on='Target',right_on='Node')
encoded_df = encoded_df.drop(['Node_x','Node_y'],axis=1)
print(merged_df.shape)
print(encoded_df.shape)


(230965, 28)
(230965, 30)


In [6]:
encoded_df.head()

,Source,Target,time_spent,id,hour,venueid,visits,venuename,address,city,...,level2cat,level3catId,level3cat,chainid,chainname,parentvenueid,zip,sample,Source_Encoding,Target_Encoding
0,2adee8895bc53ddebe8fb9c8a532f8bb5105a0d5bdcfd7...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,976,0,2,440da2cbf964a52091301fe3,18,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all,23428,12167
1,cf4e564f4c8e3aecc9dacafce90cada3eabaff58464e74...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,217,34,3,440da2cbf964a52091301fe3,5,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all,469,12167
2,d2d9fcc359e84b00df905fb373c1b7b4689e82d8105c75...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,214,18,3,440da2cbf964a52091301fe3,4,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all,2933,12167
3,f0facda51884bba89ac4eb717653078332a1eddb0ab2cb...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,526,87,4,440da2cbf964a52091301fe3,9,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all,5499,12167
4,f0facda51884bba89ac4eb717653078332a1eddb0ab2cb...,d531979041494d0d0d2387cfc611f6e70d91bcb92cf966...,202,47,4,440da2cbf964a52091301fe3,4,Austin Bergstrom International Airport (AUS),3600 Presidential Blvd,Austin,...,Airports,NaN,NaN,NaN,NaN,NaN,78719,us_all,5499,12167


In [7]:
# merged_df['level1cat'].value_counts()
encoded_df['level1cat'] = encoded_df['level1cat'].astype('category')
one_hot = pd.get_dummies(encoded_df.level1cat,prefix="cat")
encoded_df = encoded_df.join(one_hot)
encoded_df.columns

Index(['Source', 'Target', 'time_spent', 'id', 'hour', 'venueid', 'visits',
       'venuename', 'address', 'city', 'dma', 'state', 'country', 'geolat',
       'geolong', 'catid', 'catname', 'level1catId', 'level1cat',
       'level2catId', 'level2cat', 'level3catId', 'level3cat', 'chainid',
       'chainname', 'parentvenueid', 'zip', 'sample', 'Source_Encoding',
       'Target_Encoding', 'cat_Arts & Entertainment',
       'cat_Colleges & Universities', 'cat_Events', 'cat_Food',
       'cat_Nightlife Spots', 'cat_Outdoors & Recreation',
       'cat_Professional & Other Places', 'cat_Residences',
       'cat_Shops & Services', 'cat_Travel & Transport'],
      dtype='object')